In [27]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
import os
import keras
import math
import numpy as np

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
#weights_path = '../model/vgg16_weights.h5'
top_model_weights_path = '../model/bottleneck_fc_model.h5'
#top_model_weights_path = '../model/fc_model.h5'
img_width, img_height = 150, 150

#full
#train_data_dir = '../data/Pandora18K_train_val_test_split/train'
#validation_data_dir = '../data/Pandora18K_train_val_test_split/val'
#nb_train_samples = 14313
#nb_validation_samples = 1772
#total_num_classes = 18

#small
train_data_dir = '../data/Pandora18K_small_train_val_test_split/train'
validation_data_dir = '../data/Pandora18K_small_train_val_test_split/val'
nb_train_samples = 1462
nb_validation_samples = 167
total_num_classes = 18

#verysmall
#train_data_dir = '../data/verysmall/train'
#validation_data_dir = '../data/verysmall/val'
#nb_train_samples = 144 #157
#nb_validation_samples = 16 #18
#total_num_classes = 2

epochs = 2
batch_size = 16

In [28]:

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
"""
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
#top_model.add(Dropout(0.5))
top_model.add(Dense(total_num_classes, activation='sigmoid'))
"""


top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dropout(0.5))
top_model.add(Dense(1024, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(total_num_classes, activation='softmax'))



# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
# model.add(top_model)

model = Model(input= base_model.input, output= top_model(base_model.output))








Model loaded.


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:33: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`


In [29]:
best_weight_path = "../model/best.h5"
best_acc = 0

def get_model_val_acc(a_weight_path):
    #sanity check that the saved best_weight_path contains weights that gets you an accuracy of best_acc
    sanity_val_datagen = ImageDataGenerator(rescale=1. / 255)
    sanity_val_generator = sanity_val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

    model.load_weights(a_weight_path)

    sanity_loss, sanity_acc = model.evaluate_generator(generator = sanity_val_generator, steps = nb_validation_samples // batch_size)
    print("best val loss check = " + str(sanity_loss))
    print("best val acc check = " + str(sanity_acc))
    
    return sanity_acc

class AccuracyHistory(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        
        global best_acc
        current_acc = logs.get('val_acc')
        
        if current_acc > best_acc:
            print("\n\na better current_acc = " + str(current_acc) + "\n\n")
            best_acc = current_acc
            model.save_weights(best_weight_path,overwrite=True)


def run_with_parameters(frozen_layers = 25, lr = 1e-4, momentum = 0.9, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, best_acc = 0):
    # set the first 25 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    for layer in model.layers:
        layer.trainable = True;
    
    for layer in model.layers[:frozen_layers]:
        layer.trainable = False

    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=lr, momentum=momentum),
                  metrics=['accuracy'])

    # prepare data augmentation configuration
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=shear_range,
        zoom_range=zoom_range,
        horizontal_flip=horizontal_flip)

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')
    
    pretrained_model = "temp_name"
    weight_dir = "../model/%s"%pretrained_model
    if not os.path.exists(weight_dir):
        os.makedirs(weight_dir)
    param_names = "frozen_layers_" + str(frozen_layers) + "_lr_" + str(lr) + "_momentum_" + str(momentum)
    weight_path = os.path.join(weight_dir, param_names + "_best_weights%s.h5"%pretrained_model)
    checkpointer = keras.callbacks.ModelCheckpoint(filepath=weight_path, verbose=1,monitor='val_acc', save_best_only=True, save_weights_only=True)
    history = AccuracyHistory()
    
    # fine-tune the model
    model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        epochs=epochs,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples,
        callbacks=[checkpointer])
#        callbacks=[checkpointer, history])
    
    a_new_acc = get_model_val_acc(weight_path)
    if a_new_acc > best_acc:
        model.load_weights(weight_path)
        model.save_weights(best_weight_path)
        best_acc = a_new_acc
    
    #model.load_weights(weight_path)
    #validation_labels = np.array([0]*8+[1]*10+[2]*11+[3]*5+[4]*11+[5]*6+[6]*8+[7]*8+[8]*9+[9]*12+[10]*7+[11]*10+[12]*14+[13]*5+[14]*11+[15]*11+[16]*12+[17]*9)
    #loss, acc = model.evaluate(x = validation_generator, y = validation_labels, batch_size=batch_size, verbose=1, sample_weight=None)
    #if acc > best_acc:
    #    model.save_weights(weight_path)
        
    #return acc


In [30]:

frozen_layers_arr = [25]
lr_arr = [1e-4, 1e-5]
momentum_arr = [0.9]
shear_range_arr = [0.2]
zoom_range_arr = [0.2]
horizontal_flip_arr = [True]

for frozen_layers_val in frozen_layers_arr:
    for lr_val in lr_arr:
        print("\n")
        print("frozen_layers_val = " + str(frozen_layers_val))
        print("lr_val = " + str(lr_val))

        
        """
        new_weight_path = run_with_parameters(frozen_layers = frozen_layers_val, lr = lr_val)
        
        print("new_weight_path = " + new_weight_path)
        model.load_weights(new_weight_path)
        
        validation_data = np.load(open('../model/bottleneck_features_validation.npy'))
        validation_labels = np.array([0]*8+[1]*10+[2]*11+[3]*5+[4]*11+[5]*6+[6]*8+[7]*8+[8]*9+[9]*12+[10]*7+[11]*10+[12]*14+[13]*5+[14]*11+[15]*11+[16]*12+[17]*9)
        
        print(validation_data.shape)
        print(validation_labels.shape)
        new_acc = model.evaluate(x=, y=, batch_size=batch_size, verbose=1)
        print("accuracy = " + str(new_acc))
        os.rename(new_weight_path, new_weight_path + "_acc_" + str(new_acc))
        """
        
        new_acc = run_with_parameters(frozen_layers = frozen_layers_val, lr = lr_val, best_acc = best_acc)
        #if new_acc > best_acc:
        #    best_acc = new_acc
        
        #if new_acc > best_acc:
            

#print("\n\n\nbest val acc = " + str(best_acc))

#run_with_parameters(frozen_layers = 25, lr = 1e-4, momentum = 0.9, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    



frozen_layers_val = 25
lr_val = 0.0001
Found 1462 images belonging to 18 classes.
Found 167 images belonging to 18 classes.
Epoch 1/2
91/91 [==============================] - 40s - loss: 2.7672 - acc: 0.1525 - val_loss: 2.5888 - val_acc: 0.2038
Epoch 2/2
91/91 [==============================] - 39s - loss: 2.7402 - acc: 0.1719 - val_loss: 2.5861 - val_acc: 0.2034
Found 167 images belonging to 18 classes.
best val acc check = 2.5705360651
best val acc check = 0.2125


frozen_layers_val = 25
lr_val = 1e-05
Found 1462 images belonging to 18 classes.
Found 167 images belonging to 18 classes.
Epoch 1/2
91/91 [==============================] - 40s - loss: 2.8004 - acc: 0.1669 - val_loss: 2.5875 - val_acc: 0.2034
Epoch 2/2
91/91 [==============================] - 39s - loss: 2.7791 - acc: 0.1646 - val_loss: 2.5882 - val_acc: 0.2022
Found 167 images belonging to 18 classes.
best val acc check = 2.55662858486
best val acc check = 0.20625



best val acc = 0


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:85: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=91, epochs=2, callbacks=[<keras.ca..., validation_steps=167)`


In [31]:
#sanity check that the saved best_weight_path contains weights that gets you an accuracy of best_acc
sanity_val_datagen = ImageDataGenerator(rescale=1. / 255)
sanity_val_generator = sanity_val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.load_weights(best_weight_path)

sanity_loss, sanity_acc = model.evaluate_generator(generator = sanity_val_generator, steps = nb_validation_samples // batch_size)
print("best val loss check = " + str(sanity_loss))
print("best val acc check = " + str(sanity_acc))

Found 167 images belonging to 18 classes.
best val acc check = 2.5965811491
best val acc check = 0.20625


SyntaxError: 'return' outside function (<ipython-input-31-011623786f41>, line 15)